# Imputation

Sometimes our features may be missing observations, what can we do?

## What we will accomplish

In this notebook we will:
- Discuss a method to deal with missing values,
- Demonstrate that method on a penguin data set,
- Illustrate various approaches to imputation and
- Show how to integrate imputation into a train test split procedure.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from seaborn import set_style
set_style("whitegrid")

## Missing data

There will be times when you need to work with a data set that has missing data. We can see one such example with this edited `seaborn` penguins data set, <a href="https://seaborn.pydata.org/examples/scatterplot_matrix.html">https://seaborn.pydata.org/examples/scatterplot_matrix.html</a>.

In [2]:
penguins = pd.read_csv("../../data/penguins_w_nas.csv")

penguins.loc[penguins.sex=='Male', 'sex'] = 0
penguins.loc[penguins.sex=='Female', 'sex'] = 1

In [3]:
penguins.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            344 non-null    object 
 1   bill_length_mm     342 non-null    float64
 2   bill_depth_mm      341 non-null    float64
 3   flipper_length_mm  342 non-null    float64
 4   body_mass_g        339 non-null    float64
 5   sex                333 non-null    object 
dtypes: float64(4), object(2)
memory usage: 16.2+ KB


Using `.info()` above we can see that the data set has 344 entries with some observations that are missing values for one or more columns. If there are NAs in features that you plan to use in a model you will be unable to use those observations in training or validation sets.

So what can we do?

## Imputation

The process of replacing missing values in data is known as <i>imputation</i>. There are a few different ways you can impute missing data.

### Imputing a preset constant value

The simplest approach is just just impute a constant value. For example, maybe scientists have already observed an average body mass for penguins, let's say it is `4207`. You could then impute this value for all missing `body_mass_g` values.

In [4]:
## Make a copy to demonstrate impute strategy
penguins_constant_impute = penguins.copy()

## Replace the missing data
## .isna() checks for missing data
penguins_constant_impute.loc[penguins_constant_impute.body_mass_g.isna(), 'body_mass_g'] = 4207

In [5]:
penguins_constant_impute.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            344 non-null    object 
 1   bill_length_mm     342 non-null    float64
 2   bill_depth_mm      341 non-null    float64
 3   flipper_length_mm  342 non-null    float64
 4   body_mass_g        344 non-null    float64
 5   sex                333 non-null    object 
dtypes: float64(4), object(2)
memory usage: 16.2+ KB


### Imputing based on a sample statistic

A common strategy is to impute missing values using sample statistics from the non-missing values in a column. For example, we can replace NAs with the mean, median or mode of the column.

While we can do this by hand using `numpy` or `pandas` it may be easier, and more easily used in predictive modeling, to use `sklearn`'s `SimpleImputer`, <a href="https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html">https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html</a>.

In [6]:
from sklearn.impute import SimpleImputer

In [7]:
## make a SimpleImputer object
## strategy determines how the imputation happens
## options are 'mean', 'median', 'most_frequent' and 'constant'
impute = SimpleImputer(strategy = 'median')

## fit the impute object
impute.fit(penguins['body_mass_g'].values.reshape(-1,1))

## Show the transformed data
impute.transform(penguins['body_mass_g'].values.reshape(-1,1))[penguins.body_mass_g.isna()]

array([[4050.],
       [4050.],
       [4050.],
       [4050.],
       [4050.]])

In [8]:
penguins.body_mass_g.median()

4050.0

### Building a model to impute

We can also build a model to impute the missing values. Let's regress `body_mass_g` on all other columns to get imputed values for the missing data.

Note that we cannot do this for observations missing multiple features. For now we will ignore those rows, but in practice we would have to come up for unique imputation strategies for each columns (and sometimes each row).

In [9]:
penguins.loc[penguins.body_mass_g.isna()]

,species,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
3,Adelie,NaN,NaN,NaN,NaN,NaN
23,Adelie,38.2,18.1,185.0,NaN,0
32,Adelie,39.5,17.8,188.0,NaN,1
228,Gentoo,43.3,13.4,209.0,NaN,1
339,Gentoo,NaN,NaN,NaN,NaN,NaN


In [10]:
penguins_na = penguins.loc[penguins.body_mass_g.isna()].dropna(subset=['bill_length_mm', 
                                                                       'bill_depth_mm', 
                                                                       'flipper_length_mm', 
                                                                       'sex']).copy()
penguins_non_na = penguins.dropna().copy()

In [11]:
penguins_na[['Adelie', 'Gentoo']] = pd.get_dummies(penguins_na.species)[['Adelie', 'Gentoo']].copy()
penguins_non_na[['Adelie', 'Gentoo']] = pd.get_dummies(penguins_non_na.species)[['Adelie', 'Gentoo']].copy()

In [12]:
from sklearn.linear_model import LinearRegression

In [13]:
reg = LinearRegression(copy_X=True)

reg.fit(penguins_non_na[['bill_length_mm', 
                         'bill_depth_mm', 
                         'flipper_length_mm', 
                         'sex', 
                         'Adelie', 
                         'Gentoo']].values,
        penguins_non_na.body_mass_g.values)

LinearRegression()

In [14]:
reg.predict(penguins_na[['bill_length_mm', 
                         'bill_depth_mm', 
                         'flipper_length_mm', 
                         'sex', 
                         'Adelie', 
                         'Gentoo']].values)

array([3789.59016416, 3455.21140107, 4578.58391692])

For missing values that occur in rows where modeling is impossible you would have to use one of the other prior strategies.

#### A note about predictive modeling

If the end goal for your data set is to build a predictive model you <b>cannot</b> use the column that you are trying to predict in an imputation model. For example, if we were looking to build a model that predicted the `species` of the penguin we would not be allowed to use it for feature imputation models.

## Imputation in predictive modeling projects

Recall that when we scale data in predictive modeling projects we have to fit the scaler on the training data and then use that fit scaler on the test data. Importantly, we do <b>not</b> refit the scaler on the test or holdout data. We have to take a similar approach to any imputation technique.

For example:
- Imputation using any sample statistic approach must use the sample statistic computed on the training data to impute missing values in the test, validation or holdout sets and
- Models trained to impute missing values must be fit on the training data and that fitted model is what used on the test data.

`sklearn`'s `SimpleImputer` object is nicely set up for this.

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
penguins_train, penguins_test = train_test_split(penguins, 
                                                    shuffle=True,
                                                    random_state=32,
                                                    test_size=.2)

In [17]:
penguins_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 275 entries, 180 to 215
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            275 non-null    object 
 1   bill_length_mm     275 non-null    float64
 2   bill_depth_mm      274 non-null    float64
 3   flipper_length_mm  275 non-null    float64
 4   body_mass_g        273 non-null    float64
 5   sex                266 non-null    object 
dtypes: float64(4), object(2)
memory usage: 15.0+ KB


In [18]:
penguins_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69 entries, 266 to 197
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            69 non-null     object 
 1   bill_length_mm     67 non-null     float64
 2   bill_depth_mm      67 non-null     float64
 3   flipper_length_mm  67 non-null     float64
 4   body_mass_g        66 non-null     float64
 5   sex                67 non-null     object 
dtypes: float64(4), object(2)
memory usage: 3.8+ KB


In [19]:
## Define an imputer with the 'mean' strategy
impute = SimpleImputer(strategy = 'mean')



## fitting the imputer on the training set
impute.fit(penguins_train[['bill_length_mm', 
                          'bill_depth_mm', 
                          'flipper_length_mm',
                          'body_mass_g']])

## Imputing the training set
impute.transform(penguins_train[['bill_length_mm', 
                              'bill_depth_mm', 
                              'flipper_length_mm',
                              'body_mass_g']])


array([[  46.4,   17.8,  191. , 3700. ],
       [  40.5,   18.9,  180. , 3950. ],
       [  40.2,   20.1,  200. , 3975. ],
       ...,
       [  47.5,   15. ,  218. , 4950. ],
       [  44.1,   19.7,  196. , 4400. ],
       [  55.8,   19.8,  207. , 4000. ]])

In [20]:
## Imputing the test set
impute.transform(penguins_test[['bill_length_mm', 
                          'bill_depth_mm', 
                          'flipper_length_mm',
                          'body_mass_g']])

array([[  45.5       ,   13.9       ,  210.        , 4200.        ],
       [  46.        ,   21.5       ,  194.        , 4200.        ],
       [  49.6       ,   16.        ,  225.        , 5700.        ],
       [  52.2       ,   18.8       ,  197.        , 3450.        ],
       [  38.1       ,   17.6       ,  187.        , 3425.        ],
       [  45.7       ,   13.9       ,  214.        , 4400.        ],
       [  43.2       ,   19.        ,  197.        , 4775.        ],
       [  49.8       ,   17.3       ,  198.        , 3675.        ],
       [  36.4       ,   17.        ,  195.        , 3325.        ],
       [  40.8       ,   18.9       ,  208.        , 4300.        ],
       [  45.6       ,   20.3       ,  191.        , 4600.        ],
       [  38.6       ,   21.2       ,  191.        , 3800.        ],
       [  38.1       ,   17.        ,  181.        , 3175.        ],
       [  37.        ,   16.9       ,  185.        , 3000.        ],
       [  41.5       ,   18.3     

`SimpleImputer` (and `sklearn`'s other imputer objects, <a href="https://scikit-learn.org/stable/modules/classes.html#module-sklearn.impute">https://scikit-learn.org/stable/modules/classes.html#module-sklearn.impute</a>) can be implemented into a pipeline just like `StandardScaler` or any other `sklearn` preprocessing object.

--------------------------

This notebook was written for the Erd&#337;s Institute C&#337;de Data Science Boot Camp by Matthew Osborne, Ph. D., 2023.

Any potential redistributors must seek and receive permission from Matthew Tyler Osborne, Ph.D. prior to redistribution. Redistribution of the material contained in this repository is conditional on acknowledgement of Matthew Tyler Osborne, Ph.D.'s original authorship and sponsorship of the Erdős Institute as subject to the license (see License.md)